# Cognitive Radiology Report Generation - Google Colab Setup

This notebook runs the Cognitive Radiology Report Generation project. 

**Prerequisites:**
1. `project_codebase.zip` (The code we just packed)
2. `kaggle.json` OR your Kaggle API Token string.

## 1. Setup Environment

In [ ]:
# Upload the `project_codebase.zip` manually to the file explorer on the left first!
!unzip -o project_codebase.zip

In [ ]:
!pip install -r requirements.txt

## 2. Setup Kaggle Data
If you have `kaggle.json`, upload it now. If not, paste your API Token when prompted.

In [ ]:
import os
from google.colab import files
from getpass import getpass

if os.path.exists('kaggle.json'):
    !mkdir -p ~/.kaggle
    !cp kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json
    print("Loaded credentials from kaggle.json")
else:
    print("kaggle.json not found. Please paste your Kaggle API Token below.")
    print("(It starts with KGAT... copy it from the pop-up on Kaggle)")
    token = getpass('Kaggle API Token: ')
    if token:
        os.environ['KAGGLE_API_TOKEN'] = token.strip()
        print("✓ KAGGLE_API_TOKEN environment variable set.")
    else:
        print("Warning: No token provided.")

In [ ]:
# Download IU-Xray Dataset
!python scripts/download_with_hub.py

# Download MIMIC-CXR Dataset
!python scripts/download_mimic.py

## 3. Preprocessing

In [ ]:
!python scripts/preprocess_iu_xray.py
!python scripts/preprocess_mimic.py

In [ ]:
# Verify data structure
!ls -R data/processed | head -n 20

## 4. Training
We can increase batch size here since Colab GPUs (T4) have 16GB VRAM.
We are training on MIMIC-CXR (if downloaded) or IU-Xray.

In [ ]:
!python training/train.py --batch_size 16 --epochs 15 --num_workers 2 --dataset mimic_cxr

## 5. Evaluation

In [ ]:
!python evaluation/evaluate.py --checkpoint checkpoints/best_model.pth

## 6. Download Results

In [ ]:
!zip -r checkpoints.zip checkpoints/
files.download('checkpoints.zip')